Run this on `gatech-hive` \
~Running in pre-compiled env `/cvmfs/icecube.opensciencegrid.org/py3-v4.1.0/RHEL_7_x86_64/metaprojects/combo/stable/env-shell.sh` on cobalt06~ \

Get grb list from GRBWeb \
Start date and end date are aligned with ~GRECO v2.4 data~ GRECO v2.10 data

In [1]:
import numpy as np
import sqlite3
import pandas
import os

In [2]:
# Download the SQLite file from the GRBweb webpage
# os.system("wget -O /home/cjchen/2020-2021/Spring2021/greco_grb/data/grbweb/GRBweb2.sqlite https://icecube.wisc.edu/~grbweb_public/GRBweb2.sqlite")
os.system("wget -O /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/grbweb/GRBweb2.sqlite https://icecube.wisc.edu/~grbweb_public/GRBweb2.sqlite")


0

In [3]:
# db = sqlite3.connect('/home/cjchen/2020-2021/Spring2021/greco_grb/data/grbweb/GRBweb2.sqlite')
db = sqlite3.connect('/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/grbweb/GRBweb2.sqlite')
db

In [4]:
# Print the names of all the tables
table_names = pandas.read_sql_query("SELECT * from sqlite_sequence", db)
table_names


name    seq
0                    BATSE   2702
1                 BeppoSAX   1082
2                Fermi_GBM   3134
3  Fermi_GBM_precursor_Zhu    239
4   Fermi_GBM_subthreshold   4608
5                Fermi_LAT    146
6            GCN_circulars  18520
7                      IPN   2255
8                  Summary   7655
9                    Swift   1506

In [5]:
# Get numpy arrays containing the right ascension, declination, mjd, ... of the entries in the Summary table
Summary_table = pandas.read_sql_query("SELECT * from Summary", db)
Summary_table.head()

id    GRB_name GRB_name_Fermi        T0                        T0_source  \
0   1  GRB150702A   GRB150702998  86197.55                        Fermi_LAT   
1   2  GRB080120A           None  62910.00           GCN_circulars,INTEGRAL   
2   3  GRB080123A           None  15717.00          GCN_circulars,Swift Det   
3   4  GRB080121A           None  77395.00                            Swift   
4   5  GRB080122A           None  66764.00  GCN_circulars,IPN Triangulation   

           ra  ra_source       decl decl_source  pos_error  ...       fluence  \
0   52.780000  Fermi_LAT -57.000000   Fermi_LAT   0.360000  ...  1.174100e-05   
1  225.287500        IPN -10.900000         IPN   0.051667  ...           NaN   
2  338.943083      Swift -64.900889       Swift   0.000220  ...  5.700000e-07   
3  137.235000      Swift  41.841000       Swift   0.023300  ...  3.000000e-08   
4  191.350000        IPN  27.016667         IPN   1.666667  ...           NaN   

   fluence_source fluence_error  fluence_error_source redshift  \
0       Fermi_GBM  5.402800e-08             Fermi_GBM      NaN   
1            None           NaN                  None      NaN   
2           Swift           NaN                  None      NaN   
3           Swift           NaN                  None    0.046   
4            None           NaN                  None      NaN   

               redshift_source    T100  GBM_located           mjd  \
0                         None   48.17            0  57205.997657   
1                         None     NaN            0  54485.728125   
2                         None  115.00            0  54488.181910   
3  GCN_circulars,Swift-BAT Det    0.70            0  54486.895775   
4                         None     NaN            0  54487.772731   

                        mjd_source  
0                        Fermi_LAT  
1           GCN_circulars,INTEGRAL  
2          GCN_circulars,Swift Det  
3                            Swift  
4  GCN_circulars,IPN Triangulation  

[5 rows x 27 columns]

In [6]:
keys = Summary_table.keys()
print("Keys in the 'Summary' table: {}\n".format(list(keys)))
RA  = np.array(Summary_table.ra)   # right ascension
DEC = np.array(Summary_table.decl) # declination
MJD = np.array(Summary_table.mjd)  # modified julian date
print(" RA: {}".format(RA[:3]))
print(" DEC: {}".format(DEC[:3]))
print(" MJD: {}".format(MJD[:3]))

Summary_table.shape

Keys in the 'Summary' table: ['id', 'GRB_name', 'GRB_name_Fermi', 'T0', 'T0_source', 'ra', 'ra_source', 'decl', 'decl_source', 'pos_error', 'pos_error_source', 'T90', 'T90_source', 'T90_error', 'T90_error_source', 'T90_start', 'T90_start_source', 'fluence', 'fluence_source', 'fluence_error', 'fluence_error_source', 'redshift', 'redshift_source', 'T100', 'GBM_located', 'mjd', 'mjd_source']

 RA: [ 52.78       225.2875     338.94308333]
 DEC: [-57.         -10.9        -64.90088889]
 MJD: [57205.99765683 54485.728125   54488.18190972]


(7655, 27)

## GRECO v2.4
`/data/user/mlarson/combo_r129072/scripts/greco_online/skylab_dataset/greco_online/current` \
Data 2012 - 2018 \
from MJD `56043.42313825331` to `58648.87564821623` \
Or `2012-04-26 10:09:19.145 UTC` ~ `2019-06-14 21:00:56.006 UTC` \
No boundary constraint here.

## GRECO v2.10
`/data/user/mlarson/combo_r129072/scripts/greco_online/skylab_dataset/greco_online/candidate-002-p10` \
Data 2012 - 2019 \
GRL starts from MJD `56043.42452709535` and ends at `58998.82821445378` \
Or `2012-04-26 10:11:19.141 UTC` ~ `2020-05-29 19:52:37.729 UTC` \
Boundary constraints:
`t_center > 56043.42452709535 + 500./86400` and `t_center < 58998.82821445378 - 500./86400`

In [7]:
# test1 = np.load("/data/user/mlarson/combo_r129072/scripts/greco_online/skylab_dataset/greco_online/candidate-002-p10/GRL/IC86_2012.data.npy")
# test2 = np.load("/data/user/mlarson/combo_r129072/scripts/greco_online/skylab_dataset/greco_online/candidate-002-p10/GRL/IC86_2019.data.npy")
test1 = np.load("/storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10/GRL/IC86_2012.data.npy")
test2 = np.load("/storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10/GRL/IC86_2019.data.npy")

print(np.min(test1['start']))
print(np.max(test2['stop']))


56043.42452709535
58998.82821445378


In [8]:
if np.isnan(np.sum(MJD)):
    print("We have null MJD")
else:
    print("All bursts have MJD")

All bursts have MJD


All angles in degrees \
All times in days

In [9]:
query = """
SELECT GRB_name AS grb_name
, GRB_name_Fermi AS grb_name_GBM
, ra
, decl AS dec
, mjd AS mjd_start
, T100/86400. AS t100
, T90/86400. AS t90
, GBM_located AS gbm
, pos_error AS source_uncertainty
, fluence
, redshift
, (mjd + 0.5*T100/86400.) AS t_center
FROM Summary
WHERE (T100 IS NOT NULL) And (t_center between 56043.43031413238 And 58998.822427416744)
ORDER BY grb_name DESC
"""
grb_table_2297 = pandas.read_sql_query(query, db)
grb_table_2297.head()

grb_name  grb_name_GBM          ra        dec     mjd_start      t100  \
0  GRB200529B  GRB200529504  337.200000  55.230000  58998.503432  0.000539   
1  GRB200529A  GRB200529039  238.763917 -11.073639  58998.039294  0.001004   
2  GRB200528A  GRB200528436  176.637458  58.192583  58997.435683  0.000700   
3  GRB200525A  GRB200525611   20.610000  -8.520000  58994.611328  0.001081   
4  GRB200524A  GRB200524211  212.750000  60.966667  58993.211115  0.000445   

        t90  gbm  source_uncertainty   fluence  redshift      t_center  
0  0.000539    1            3.506136  0.000005       NaN  58998.503701  
1  0.000984    0            0.000233  0.000005       NaN  58998.039796  
2  0.000655    0            0.000194  0.000029       NaN  58997.436033  
3  0.001081    1            4.536708  0.000009       NaN  58994.611869  
4  0.000437    0            0.200000  0.000014     1.256  58993.211338

In [10]:
grb_table_2297.shape

(2297, 12)

In [11]:
type(grb_table_2297)

pandas.core.frame.DataFrame

In [12]:
# grb_table.to_pickle("/home/cjchen/2020-2021/Spring2021/greco_grb/data/grbweb/grb_grbweb.pkl")
# grb_table_2297.to_pickle("/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/grbweb/grb_grbweb_2297.pkl")
